# Data Analsysis Project

#### This is an excerpt from my Data Analysis final project, fall semester of 2020 at GWU. The project consisted of answering 30 provided questions involving exoneree compensation from an Innocence Project dataset, as well as an open ended portion to be determined by the student. The dataset has nearly 2000 observations (individual exonerees), and nearly 70 inputs (race, sex, date of incarceration, incarceration time, ect...).

#### I chose to use ML  methods to predict whether an exoneree would be successful in their petition for compensation, as well inference to determine a less biased formula for determining the amount of compenstation. The course used SAS for much of the ML based analysis, though I chose to preform data cleaning and regression (logit) analysis in Python. I have rewritten a portion of the SAS code in Python for work sample purposes. This section is under the "Sample ML" section.

In [2]:
# importing modules
import pandas as pd
from pandas import ExcelWriter
import numpy as np
from scipy import stats
import os

import matplotlib.pyplot as plt

import seaborn as sns

In [19]:
# funtion
def multiple_dfs(df_list, sheets, file_name, spaces):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    row = 0
    for dataframe in df_list:
        dataframe.to_excel(writer,sheet_name=sheets,startrow=row , startcol=0)   
        row = row + len(dataframe.index) + spaces + 1
    writer.save()

## Data Cleaning and Formatting

In [3]:
# import data
df_master = pd.read_excel(
    '/Users/emilekaldany/Google Drive/Archives/School/GWU/Fall 2020/Data Analysis/Project/NRE_6210_S2018_v2.xlsx')

In [4]:
df = df_master

# cleaning data
# change messy column names
df = df.rename(columns={'State Award?': 'State Award',
                        'State Claim Made?': 'State Claim',
                        'Award via Settlement or Verdict for P': 'Civil Award',
                        'Non-Statutory Case Filed?': 'Civil Claim',
                        'Worst Crime ': 'Worst Crime',
                        'Death Penalty?':'Death Penalty',
                        'State Statute?':'State Statute', 
                        'DNA only':'DNA',
                        'No Time?': 'No Time',
                        'Age on Date of Crime': 'Crime Age'})

# race cleaning
df['Race'] = df['Race'].str.replace('caucasian', 'Caucasian')
df['Race'] = df['Race'].str.replace('Back', 'Black')

# race interaction
# asian is default
dfI = pd.get_dummies(df['Race'],drop_first=True)
df = pd.concat([dfI, df], axis=1)

# state statute cleaning
df['State Statute'] = df['State Statute'].str.upper()
df['State Statute'] = df['State Statute'].str[:1]
df['State Statute'] = df['State Statute'].replace('Y', 1)
df['State Statute'] = df['State Statute'].replace('N', 0)

# state award cleaning
df['State Award'] = df['State Award'].replace('Y', 1)
df['State Award'] = df['State Award'].replace('N', 0)
df['State Award'] = df['State Award'].replace('pending', 2)
df['State Award'] = df['State Award'].replace('premature',2)

# ciu
#df['CIU'] = df['CIU']

# decade variable
# create a list of our conditions
decades = [
    (df['Occurred'] < 1960),
    (df['Occurred'] >= 1960) & (df['Occurred'] < 1970),
    (df['Occurred'] >= 1970) & (df['Occurred'] < 1980),
    (df['Occurred'] >= 1980) & (df['Occurred'] < 1990),
    (df['Occurred'] >= 1990) & (df['Occurred'] < 2000),
    (df['Occurred'] >= 2000) & (df['Occurred'] < 2010),
    (df['Occurred'] >= 2010) & (df['Occurred'] < 2020),
    ]

# create a list of the values we want to assign for each condition
values = ['1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s']

# create a new column and use np.select to assign values to it using our lists as arguments
df['Decade'] = np.select(decades, values)

# time lost variable
# create a list of our conditions
time_lost = [
    (df['Years Lost'] == 0),
    (df['Years Lost'] > 0) & (df['Years Lost']  <= 1),
    (df['Years Lost'] > 1) & (df['Years Lost']  <= 5),
    (df['Years Lost'] > 5) & (df['Years Lost']  <= 10),
    (df['Years Lost'] > 10) & (df['Years Lost'] <= 20),
    (df['Years Lost'] > 20) & (df['Years Lost'] <= 30),
    (df['Years Lost'] > 30)
    ]

# create a list of the values we want to assign for each condition
tlvalues = ['0 Years', '0 to 1 Years', '1 to 5 Years', '5 to 10 Years', '10 to 20 Years', '20 to 30 Years',
            '30 Years or More']
# create a new column and use np.select to assign values to it using our lists as arguments
df['Time Lost'] = np.select(time_lost, tlvalues)

# allow division and create Civil award per time lost
df['Years Lost'] = pd.to_numeric(df['Years Lost'])
df['Civil Amount'] = pd.to_numeric(df['Civil Amount'] , errors='coerce')

df['Civil Amount Per Year'] = df['Civil Amount'].divide(df['Years Lost'].where(
    df['Years Lost']!= 0))


# geographic area
# create a list of our conditions
East = ['Maine', 'New Hampshire', 'Vermont','Massachusetts', 'Rhode Island', 'Connecticut','New York', 
        'Pennsylvania', 'New Jersey']
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico','Alaska', 
        'Washington', 'Oregon', 'California', 'Hawaii']
South= ['Oklahoma', 'Texas', 'Arkansas', 'Louisiana','Kentucky', 'Tennessee', 'Mississippi', 'Alabama',
        'Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 
        'South Carolina', 'Georgia', 'Florida']
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio','Missouri', 'North Dakota', 'South Dakota', 
           'Nebraska', 'Kansas', 'Minnesota', 'Iowa']

df['Region'] = np.where(df['State'].isin(East), 'East',
               np.where(df['State'].isin(West), 'West', 
               np.where(df['State'].isin(South), 'South',
               np.where(df['State'].isin(Midwest), 'Midwest',df['State']))))

dfI = pd.get_dummies(df['Region'],drop_first=True)
df = pd.concat([dfI, df], axis=1)

# election 
# create a list of our conditions
Clinton = ['Maine', 'New Hampshire', 'Vermont','Massachusetts', 'Rhode Island', 'Connecticut','New York',
           'Illinois' ,'New Jersey','Washington', 'Oregon', 'California', 'Hawaii','District of Columbia',
           'Delaware', 'Maryland','Virginia','Minnesota','New Mexico', 'Colorado', 'Nevada']
Trump = ['Idaho','Pennsylvania', 'Montana', 'Wyoming', 'Utah', 'Arizona',
         'Alaska','Oklahoma', 'Texas', 'Arkansas', 'Louisiana','Kentucky', 'Tennessee', 'Mississippi',
         'Alabama','West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida','Wisconsin',
         'Michigan', 'Indiana', 'Ohio','Missouri', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 
         'Iowa']

df['Election'] = np.where(df['State'].isin(Clinton), 'Democrat', 
                          np.where(df['State'].isin(Trump), 'Republican', 'Error'))

dfI = pd.get_dummies(df['Election'],drop_first=True)
df = pd.concat([dfI, df], axis=1)


# tags
df['Tags'] = df['Tags'].str.upper()
sp = df['Tags'].str.get_dummies(sep=(';|,|:|#| '))
split = pd.DataFrame(sp)
split = split.rename(columns={'CIU' : 'CIU2'})
df = pd.concat([df, split], axis=1, sort=False)

df

,Error,Republican,Guam,Midwest,Puerto Rico,South,West,Black,Caucasian,Hispanic,...,CV,F,H,JI,M,NC,P,PH,SBS,|
0,0,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,0,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1896,0,1,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1897,0,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1898,0,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# Sample

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

NameError: name 'numerical_cols' is not defined

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

In [ ]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

In [ ]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

# Exploratory Statisitcs and Tables

In [22]:
# tables
dataframes = []

for i in ['Race', 'Sex', 'CIU', 'Guilty Plea', 'IO', 'DNA','FC','MWID','F/MFE','P/FA','OM','ILD',
          'State Statute','State Claim','State Award','Civil Claim', 'Civil Award','A','CDC','CIU2',
          'CSH','CV','F','H','JI','M','NC','P','PH','SBS', 'Region', 'Election', 'No Time']:
# counts
    
    count = pd.DataFrame(df[i].value_counts())
    percent = pd.DataFrame(count.divide(count[i].sum()))
    freqtable = pd.concat([count,percent],axis=1)
    
    #save to a list
    dataframes.append(freqtable)
    


# save to an excel sheet
multiple_dfs(dataframes, 'Validation', 'simplefreqtables', 1)
    



In [23]:

    for j in ['State Claim','State Award','Civil Claim', 'Civil Award']:
        for i in ['Race', 'Sex', 'CIU', 'Guilty Plea', 'IO', 'DNA','FC','MWID','F/MFE','P/FA','OM','ILD',
          'A','CDC','CIU2','CSH','CV','F','H','JI','M','NC','P','PH','SBS', 'Region', 'Election', 'No Time']:
        count = pd.crosstab(df[i], df[j])
        percent = pd.crosstab(df[i],df[j], normalize = 'index')
        freqtable = pd.concat([count,percent],axis=1)
        print(freqtable)
        
        
        #save to a list
        dataframes.append(freqtable)
    


# save to an excel sheet
multiple_dfs(dataframes, 'Validation', 'complexfreqtables', 1)


State Claim      0.0  1.0       0.0       1.0
Race                                         
Asian              7    4  0.636364  0.363636
Black            378  459  0.451613  0.548387
Caucasian        379  225  0.627483  0.372517
Hispanic         122   85  0.589372  0.410628
Native American    3    4  0.428571  0.571429
Other              7    1  0.875000  0.125000
State Award      0.0  1.0  2.0       0.0       1.0       2.0
Race                                                        
Asian              6    3    2  0.545455  0.272727  0.181818
Black            355  366  124  0.420118  0.433136  0.146746
Caucasian        393  135   77  0.649587  0.223140  0.127273
Hispanic         123   70   14  0.594203  0.338164  0.067633
Native American    4    2    1  0.571429  0.285714  0.142857
Other              5    1    2  0.625000  0.125000  0.250000
Civil Claim      0.0  1.0       0.0       1.0
Race                                         
Asian              9    4  0.692308  0.307692
Black 

State Award  0.0  1.0  2.0       0.0       1.0       2.0
A                                                       
0            883  576  219  0.526222  0.343266  0.130513
1              3    1    1  0.600000  0.200000  0.200000
Civil Claim   0.0  1.0       0.0       1.0
A                                         
0            1105  789  0.583421  0.416579
1               4    1  0.800000  0.200000
Civil Award     0    1         0         1
A                                         
0            1452  443  0.766227  0.233773
1               4    1  0.800000  0.200000
State Claim  0.0  1.0       0.0       1.0
CDC                                      
0            890  767  0.537115  0.462885
1              6   11  0.352941  0.647059
State Award  0.0  1.0  2.0       0.0       1.0       2.0
CDC                                                     
0            879  568  219  0.527611  0.340936  0.131453
1              7    9    1  0.411765  0.529412  0.058824
Civil Claim   0.0  1.0       0.0

Civil Award     0    1         0         1
SBS                                       
0            1456  444  0.766316  0.233684
State Claim  0.0  1.0       0.0       1.0
Region                                   
East         100  226  0.306748  0.693252
Midwest      207  225  0.479167  0.520833
South        419  247  0.629129  0.370871
West         170   80  0.680000  0.320000
State Award  0.0  1.0  2.0       0.0       1.0       2.0
Region                                                  
East         137  153   36  0.420245  0.469325  0.110429
Midwest      187  169   77  0.431871  0.390300  0.177829
South        374  210   90  0.554896  0.311573  0.133531
West         188   45   17  0.752000  0.180000  0.068000
Civil Claim  0.0  1.0       0.0       1.0
Region                                   
East         180  210  0.461538  0.538462
Guam           1    0  1.000000  0.000000
Midwest      206  263  0.439232  0.560768
Puerto Rico    2    4  0.333333  0.666667
South        554  167  0.

In [10]:
try:
            stat, p, dof, expected = stats.chi2_contingency(freqtable)
            chitable = pd.DataFrame({'stat':[stat], 'p-value':[p], 'df':[dof]})
            print('ChiSquare\n', chitable)

            oddsratio, pvalue = stats.fisher_exact(freqtable)
            statstable = pd.DataFrame({'OR':[oddsratio], 'p-value':[pvalue]})
            print('Fisher Exact',statstable)
        except:
            continue

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 9)

In [77]:
# independence
freqtable = pd.crosstab(df['Sex'], df['State Claim']) 
freqtable

stat, p, dof, expected = stats.chi2_contingency(freqtable)
chitable = pd.DataFrame({'stat':[stat], 'p-value':[p], 'df':[dof]})
print('ChiSquare\n', chitable)

oddsratio, pvalue = stats.fisher_exact(freqtable)
statstable = pd.DataFrame({'OR':[oddsratio], 'p-value':[pvalue]})
print('Fisher Exact',statstable)

dataframes = ()
dataframes = (freqtable,chitable)
dataframes += (statstable)
dataframes

ChiSquare
         stat       p-value  df
0  30.467743  3.394644e-08   1
Fisher Exact          OR       p-value
0  2.798329  1.281819e-08


,OR,p-value
0,State Claim 0.0 1.0 Sex ...,stat p-value df 0 30.467743 ...


In [64]:
print(df['Civil Amount'], df['Years Lost'], df['Civil Amount Per Year'])


#counts
#count = df['No Time?'].value_counts()
#print(count)

#mean = df['Civil Amount Per Year'].mean()
#print(mean)

#meanrace = df.groupby('Race')['Civil Amount Per Year'].mean()
#print(meanrace)

0             0.0
1             0.0
2             0.0
3             0.0
4             0.0
          ...    
1895          0.0
1896          0.0
1897          0.0
1898          0.0
1899    1250000.0
Name: Civil Amount, Length: 1900, dtype: float64 0        1.7
1        0.1
2       19.5
3        0.0
4        2.6
        ... 
1895     1.5
1896     3.1
1897     2.9
1898    23.6
1899     2.0
Name: Years Lost, Length: 1900, dtype: float64 0            0.0
1            0.0
2            0.0
3            NaN
4            0.0
          ...   
1895         0.0
1896         0.0
1897         0.0
1898         0.0
1899    625000.0
Name: Civil Amount Per Year, Length: 1900, dtype: float64


KeyError: 'No Time?'

# Provided Questions

# C1 and C2

In [35]:
dfc1 = df[df['State Award']!=2]

sclaim_race = pd.crosstab(df['Race'], df['State Claim'],
                         margins=True, margins_name = 'Total', normalize = 'index') 
sclaim_race

saward_race = pd.crosstab(dfc1['Race'], dfc1['State Award'],
                         margins=True, margins_name = 'Total', normalize = 'index') 
saward_race

sclaim_race_decade = pd.crosstab(df['Race'], [df['Decade'],df['State Claim']],
                                 dropna = False)
sclaim_race_decade

saward_race_decade = pd.crosstab(dfc1['Race'], [dfc1['Decade'],dfc1['State Award']],
                               dropna = False)
saward_race_decade

sclaim_race_time = pd.crosstab(df['Race'], [df['Time Lost'],df['State Claim']],
                                 dropna = False)
sclaim_race_time

saward_race_time = pd.crosstab(dfc1['Race'], [dfc1['Time Lost'],dfc1['State Award']],
                                 dropna = False)
saward_race_time

Time Lost       0 Years       0 to 1 Years       1 to 5 Years         \
State Award           0  1  2            0  1  2            0   1  2   
Race                                                                   
Asian                 2  0  0            0  0  0            2   1  0   
Black                55  0  0           47  6  0           79  49  0   
Caucasian            69  0  0           44  6  0          124  21  0   
Hispanic             35  1  0           11  3  0           35  15  0   
Native American       1  0  0            0  0  0            1   0  0   
Other                 1  0  0            0  0  0            2   1  0   

Time Lost       10 to 20 Years  ...    20 to 30 Years        30 Years or More  \
State Award                  0  ...  2              0   1  2                0   
Race                            ...                                             
Asian                        1  ...  0              0   0  0                0   
Black                       81  ...  1             26  67  0                5   
Caucasian                   69  ...  0             24  20  0                2   
Hispanic                    10  ...  0              2   6  0                0   
Native American              2  ...  0              0   0  0                0   
Other                        1  ...  0              0   0  0                0   

Time Lost              5 to 10 Years         
State Award       1  2             0   1  2  
Race                                         
Asian             0  0             1   1  0  
Black            14  0            62  77  0  
Caucasian         1  0            61  31  0  
Hispanic          1  0            30  13  0  
Native American   0  0             0   1  0  
Other             0  0             1   0  1  

[6 rows x 21 columns]

In [88]:
cclaim_race = pd.crosstab(df['Race'], df['Civil Claim'], 
                          margins=True, margins_name = 'Total', normalize = 'index') 
cclaim_race

caward_race = pd.crosstab(df['Race'], df['Civil Award'],
                         margins=True, margins_name = 'Total', normalize = 'index') 
caward_race

cclaim_race_decade = pd.crosstab(df['Race'], [df['Decade'],df['Civil Claim']],
                                dropna = False)
cclaim_race_decade

caward_race_decade = pd.crosstab(df['Race'], [df['Decade'],df['Civil Award']],
                                dropna = False)
caward_race_decade

cclaim_race_time = pd.crosstab(df['Race'], [df['Time Lost'],df['Civil Claim']],
                                 dropna = False)
cclaim_race_time

caward_race_time = pd.crosstab(df['Race'], [df['Time Lost'],df['Civil Award']],
                                 dropna = False)
caward_race_time


Time Lost       0 Years    0 to 1 Years    1 to 5 Years     10 to 20 Years  \
Civil Award           0  1            0  1            0   1              0   
Race                                                                         
Asian                 2  0            2  0            1   2              1   
Black                50  6           89  5          137  24            186   
Caucasian            79  6           59  5          171  31            111   
Hispanic             35  2           16  4           44  13             26   
Native American       1  1            0  0            1   0              7   
Other                 1  0            1  0            3   0              1   

Time Lost            20 to 30 Years     30 Years or More    5 to 10 Years      
Civil Award        1              0   1                0  1             0   1  
Race                                                                           
Asian              2              1   0                0  0             2   0  
Black            106             79  40               14  6           129  50  
Caucasian         53             42  17                4  2           110  33  
Hispanic          19             10   2                0  1            37  13  
Native American    0              0   0                0  0             2   0  
Other              1              0   0                0  0             2   0

In [89]:
cclaim_race_time

Time Lost       0 Years     0 to 1 Years     1 to 5 Years     10 to 20 Years  \
Civil Claim         0.0 1.0          0.0 1.0          0.0 1.0            0.0   
Race                                                                           
Asian                 2   0            2   0            1   2              1   
Black                48   8           84  10          111  50            115   
Caucasian            70  15           58   6          144  58             73   
Hispanic             30   7           12   8           36  21             17   
Native American       1   1            0   0            1   0              3   
Other                 1   0            1   0            3   0              1   

Time Lost            20 to 30 Years     30 Years or More     5 to 10 Years      
Civil Claim      1.0            0.0 1.0              0.0 1.0           0.0 1.0  
Race                                                                            
Asian              2              1   0                0   0             2   0  
Black            177             46  73                5  15            90  89  
Caucasian         90             24  35                1   5            86  57  
Hispanic          28              6   6                0   1            29  21  
Native American    4              0   0                0   0             2   0  
Other              1              0   0                0   0             2   0

## C3

In [37]:
dfc3 = df[(df[['Civil Amount Per Year','Race','Decade','Civil Award']]['Civil Award']==1)]

c3 = pd.DataFrame(dfc3.groupby('Race', dropna = False).mean()['Civil Amount Per Year'])
c3

c3a = pd.DataFrame(dfc3.groupby(['Race','Decade'], dropna = False).mean()['Civil Amount Per Year'])
c3a

Civil Amount Per Year
Race            Decade                       
Asian           1980s           145241.586910
                1990s           312500.000000
                2000s                     NaN
Black           1960s            94314.323198
                1970s           338140.188387
                1980s           283455.501496
                1990s           433606.863038
                2000s           368816.549891
                2010s            95168.067227
Caucasian       1950s            97222.222222
                1960s           901654.172006
                1970s           104018.029919
                1980s           301405.208177
                1990s           277483.877116
                2000s           372728.463787
                2010s            26786.428571
Hispanic        1980s           438842.101199
                1990s           678232.885033
                2000s           231393.508894
                2010s           557017.543860
Native American 1970s                     NaN
Other           1990s              403.225806

## C4 and C5

In [86]:
dfc1 = df[df['State Award']!=2]

sclaim_sex = pd.crosstab(df['Sex'], df['State Claim'],
                         margins=True, margins_name = 'Total', normalize = 'index') 

saward_sex = pd.crosstab(dfc1['Sex'], dfc1['State Award'],
                         margins=True, margins_name = 'Total', normalize = 'index') 
sclaim_sex_decade = pd.crosstab(df['Sex'], [df['Decade'],df['State Claim']],
                                 dropna = False)

saward_sex_decade = pd.crosstab(dfc1['Sex'], [dfc1['Decade'],dfc1['State Award']],
                               dropna = False)

sclaim_sex_time = pd.crosstab(df['Sex'], [df['Time Lost'],df['State Claim']],
                                 dropna = False)

saward_sex_time = pd.crosstab(dfc1['Sex'], [dfc1['Time Lost'],dfc1['State Award']],
                                 dropna = False)

cclaim_sex = pd.crosstab(df['Sex'], df['Civil Claim'], 
                          margins=True, margins_name = 'Total', normalize = 'index') 

caward_sex = pd.crosstab(df['Sex'], df['Civil Award'],
                         margins=True, margins_name = 'Total', normalize = 'index') 

cclaim_sex_decade = pd.crosstab(df['Sex'], [df['Decade'],df['Civil Claim']],
                                dropna = False)

caward_sex_decade = pd.crosstab(df['Sex'], [df['Decade'],df['Civil Award']],
                                dropna = False)

cclaim_sex_time = pd.crosstab(df['Sex'], [df['Time Lost'],df['Civil Claim']],
                                 dropna = False)

caward_sex_time = pd.crosstab(df['Sex'], [df['Time Lost'],df['Civil Award']],
                                 dropna = False)


In [91]:
sclaim_sex
saward_sex

cclaim_sex
caward_sex

sclaim_sex_decade
saward_sex_decade

cclaim_sex_decade
caward_sex_decade

sclaim_sex_time
saward_sex_time

cclaim_sex_time
caward_sex_time

Time Lost   0 Years    0 to 1 Years     1 to 5 Years     10 to 20 Years       \
Civil Award       0  1            0   1            0   1              0    1   
Sex                                                                            
Female           44  6           26   1           42   8             16    6   
Male            124  9          141  13          315  62            316  175   

Time Lost   20 to 30 Years     30 Years or More    5 to 10 Years      
Civil Award              0   1                0  1             0   1  
Sex                                                                   
Female                   3   1                1  0            21   5  
Male                   129  58               17  9           261  91

## C6

In [92]:
dfc6 = df[(df[['Civil Amount Per Year','Sex','Decade','Civil Award']]['Civil Award']==1)]

c6 = pd.DataFrame(dfc6.groupby('Sex', dropna = False).mean()['Civil Amount Per Year'])

c6a = pd.DataFrame(dfc6.groupby(['Sex','Decade'], dropna = False).mean()['Civil Amount Per Year'])

In [94]:
c6
c6a

Civil Amount Per Year
Sex    Decade                       
Female 1970s           323607.864293
       1980s           284114.002976
       1990s           197978.720538
       2000s           308922.558923
       2010s                     NaN
Male   1950s            97222.222222
       1960s           632540.889070
       1970s           304531.892621
       1980s           301494.901158
       1990s           447232.271678
       2000s           364288.642980
       2010s           314695.865767

In [67]:
# regions
cclaim_region = pd.DataFrame(pd.crosstab(df['Region'], df['Civil Claim'], 
                          margins=True, margins_name = 'Total', normalize = 'index'))
cclaim_region

caward_region = pd.DataFrame(pd.crosstab(df['Region'], df['Civil Award'],
                         margins=True, margins_name = 'Total', normalize = 'index')) 
caward_region

sclaim_region = pd.DataFrame(pd.crosstab(df['Region'], df['State Claim'], 
                          margins=True, margins_name = 'Total', normalize = 'index'))
sclaim_region


dfc8= df.loc[df['State Award'].isin([0, 1])]
saward_region = pd.DataFrame(pd.crosstab(dfc8['Region'], dfc8['State Award'], 
                          margins=True, margins_name = 'Total', normalize = 'index'))
saward_region

State Award,0.0,1.0
Region,,
East,0.472414,0.527586
Midwest,0.525281,0.474719
South,0.640411,0.359589
West,0.806867,0.193133
Total,0.605605,0.394395


In [64]:
# election
cclaim_election = pd.DataFrame(pd.crosstab(df['Election'], df['Civil Claim'], 
                          margins=True, margins_name = 'Total', normalize = 'index'))

caward_election = pd.DataFrame(pd.crosstab(df['Election'], df['Civil Award'],
                         margins=True, margins_name = 'Total', normalize = 'index')) 

sclaim_election = pd.DataFrame(pd.crosstab(df['Election'], df['State Claim'], 
                          margins=True, margins_name = 'Total', normalize = 'index'))


dfc8= df.loc[df['State Award'].isin([0, 1])]
saward_election = pd.DataFrame(pd.crosstab(dfc8['Election'], dfc8['State Award'], 
                          margins=True, margins_name = 'Total', normalize = 'index'))

In [68]:
cclaim_election
caward_election
sclaim_election
saward_election

State Award,0.0,1.0
Election,,
Clinton,0.571053,0.428947
Trump,0.642959,0.357041
Total,0.605605,0.394395


## C15